In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

dataset_dir = "archive/train_test"

def preprocess_image(image_path, target_size=(384, 384)):
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize(target_size)
    preprocessed_image = np.array(image) / 255.0
    return preprocessed_image

def is_authentic(filename):
    prefix = filename[:2]
    return (prefix == "au" or prefix == "Au")

images = []
labels = []

for filename in os.listdir(dataset_dir):
    if filename.startswith(".DS_Store"):
        continue
    file_path = os.path.join(dataset_dir, filename)
    preprocessed_image = preprocess_image(file_path)
    images.append(preprocessed_image)
    label = is_authentic(filename)
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

data_augmentation = ImageDataGenerator(
    rotation_range = 25,
    height_shift_range = 0.25,
    width_shift_range = 0.25,
    vertical_flip = True,
    horizontal_flip = True,
    zoom_range = 0.25
)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model architecture
base = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=train_images.shape[1:])

x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=base.input, outputs=output)

# Compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

results = model.fit(data_augmentation.flow(train_images, train_labels, batch_size=32), epochs=20,
                    steps_per_epoch=len(train_images) // 32,
                    validation_data=(test_images, test_labels))

Metal device set to: Apple M1 Max
Epoch 1/20


2023-04-13 16:20:07.781913: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


196/196 [==============================] - 295s 1s/step - loss: 0.5281 - accuracy: 0.7288 - val_loss: 0.4162 - val_accuracy: 0.8027
Epoch 2/20
196/196 [==============================] - 281s 1s/step - loss: 0.3968 - accuracy: 0.8183 - val_loss: 0.3768 - val_accuracy: 0.8211
Epoch 3/20
196/196 [==============================] - 271s 1s/step - loss: 0.3274 - accuracy: 0.8507 - val_loss: 0.2776 - val_accuracy: 0.8674
Epoch 4/20
196/196 [==============================] - 269s 1s/step - loss: 0.2851 - accuracy: 0.8671 - val_loss: 0.2657 - val_accuracy: 0.8699
Epoch 5/20
196/196 [==============================] - 266s 1s/step - loss: 0.2576 - accuracy: 0.8847 - val_loss: 0.2631 - val_accuracy: 0.8687
Epoch 6/20
196/196 [==============================] - 278s 1s/step - loss: 0.2333 - accuracy: 0.8933 - val_loss: 0.2995 - val_accuracy: 0.8604
Epoch 7/20
196/196 [==============================] - 269s 1s/step - loss: 0.2131 - accuracy: 0.8966 - val_loss: 0.2470 - val_accuracy: 0.8845
Epoch 8/20

In [2]:
# Convert and save the model as a TensorFlow Lite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('manipulation_detector_config_4.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/x9/lnqw3ctd65g1t8sjc2w_k5hm0000gn/T/tmpj05ud2mc/assets


INFO:tensorflow:Assets written to: /var/folders/x9/lnqw3ctd65g1t8sjc2w_k5hm0000gn/T/tmpj05ud2mc/assets
2023-04-13 17:52:37.449921: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-13 17:52:37.450255: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


In [3]:
# Evaluate the model on the testing set and print the accuracy
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)

50/50 [==============================] - 11s 200ms/step - loss: 0.2932 - accuracy: 0.8959
Test accuracy: 0.8959391117095947
